In [4]:
import pandas as pd
from pyspark.sql import SparkSession

In [48]:
# "Employee":["id","salary"]},"rows":{"Employee":[[1,100],[2,200],[3,300]]
pd.DataFrame([[1,100],[2,200],[3,200],[4,300]], columns=["id","salary"]).to_csv("./Employee.txt", index=None)

In [7]:
spark = SparkSession.builder.\
        appName('local[*]').\
        config("spark.sql.shuffle.partition","4").\
        getOrCreate()

In [49]:
employee = spark.read.option("header", True).csv("./Employee.txt")
employee.show()

+---+------+
| id|salary|
+---+------+
|  1|   100|
|  2|   200|
|  3|   200|
|  4|   300|
+---+------+



Write an SQL query to report the second highest salary from the Employee table. If there is no second highest salary, the query should report null.


In [71]:
from pyspark.sql.window import Window	

from pyspark.sql.functions import rank	

from pyspark.sql.functions import row_number, col

def pipeline(df):
    Window_Spec  = Window.orderBy(col("salary").desc())	


    df.withColumn("row_number",row_number().over(Window_Spec))\
            .show(truncate=False)


    # Defining rank() function

    e = df.withColumn("rank",rank().over(Window_Spec))
    e = e.select(e['salary'].alias("second highest salary")).where(e.rank==2).distinct()
    e.select("second highest salary").show()


In [67]:
pipeline(employee)

22/07/08 23:28:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+------+----------+
|id |salary|row_number|
+---+------+----------+
|4  |300   |1         |
|2  |200   |2         |
|3  |200   |3         |
|1  |100   |4         |
+---+------+----------+



22/07/08 23:28:06 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---------------------+
|second highest salary|
+---------------------+
|                  200|
+---------------------+



In [72]:
pd.DataFrame([[1,100]], columns=["id","salary"]).to_csv("./Employee1.txt", index=None)
employee1 = spark.read.option("header", True).csv("./Employee1.txt")
employee1.show()
pipeline(employee1)

+---+------+
| id|salary|
+---+------+
|  1|   100|
+---+------+



22/07/08 23:29:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---+------+----------+
|id |salary|row_number|
+---+------+----------+
|1  |100   |1         |
+---+------+----------+



22/07/08 23:29:43 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.


+---------------------+
|second highest salary|
+---------------------+
+---------------------+

